In [2]:
import pandas as pd
import numpy
import os
from torchtext import datasets
from torchtext.legacy import data

- In this competition, you’ll identify specific clinical concepts in patient notes. 
- Specifically, you'll develop an automated method to map clinical concepts from an exam rubric (e.g., “diminished appetite”) to various ways in which these concepts are expressed in clinical patient notes written by medical students (e.g., “eating less,” “clothes fit looser”). Great solutions will be both accurate and reliable.

- If successful, you'll help tackle the biggest practical barriers in patient note scoring, making the approach more transparent, interpretable, and easing the development and administration of such assessments. As a result, medical practitioners will be able to explore the full potential of patient notes to reveal information relevant to clinical skills assessment.

In [3]:
train = pd.read_table('/Users/suhancho/data/kaggle/nbme/train.csv',sep=',')
test = pd.read_table('/Users/suhancho/data/kaggle/nbme/test.csv',sep=',')
features = pd.read_table('/Users/suhancho/data/kaggle/nbme/features.csv',sep=',')
notes = pd.read_table('/Users/suhancho/data/kaggle/nbme/patient_notes.csv',sep=',')
sub = pd.read_table('/Users/suhancho/data/kaggle/nbme/sample_submission.csv',sep=',')

In [4]:
test_merged = pd.merge(test,features,on=['case_num','feature_num'])

In [12]:
merged = pd.merge(train,features,on=['case_num','feature_num'])

In [13]:
# merged['annotation'] = merged['annotation'].str.replace("[]",'1000')
merged['annotation'] = merged['annotation'].str.replace('[','').str.replace(']','')
merged['annotation'] = merged['annotation'].str.replace("'",'')
# merged = merged.replace(r'', 1000)

/var/folders/3m/68621g4n7pd2p55cvf4c9tmh0000gn/T/ipykernel_2593/123955389.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged['annotation'] = merged['annotation'].str.replace('[','').str.replace(']','')


In [14]:
str(notes[(notes.pn_num==16) & (notes.case_num==0)]['pn_history'].tolist()[0])[696:724]

'dad with recent heart attcak'

In [15]:
str(notes[(notes.pn_num==16) & (notes.case_num==0)]['pn_history'].tolist()[0])

'HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with "thyroid disease," dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms'

In [16]:
merged = merged[['annotation','feature_num']]

In [17]:
merged = merged[merged['annotation']!='']

In [18]:
merged['annotation'] = merged['annotation'].str.replace(':','').str.replace('-',' ')

In [19]:
merged

,annotation,feature_num
0,dad with recent heart attcak,0
2,father heart attack,0
3,Father MI,0
4,Dad MI,0
5,father had acute MI,0
...,...,...
14292,feeling warm,916
14295,subjective fever,916
14297,feels warm,916
14298,Felt warm,916


In [36]:
merged.to_csv('/Users/suhancho/data/kaggle/nbme/train.tsv',sep='\t',index=None)

In [81]:
len(set(merged['feature_num']))

143

In [251]:
notes_csv = notes[['pn_history']]
notes_csv['pn_history'] = notes_csv['pn_history'].str.replace(':','').str.replace('-',' ').str.replace(',',' ').str.replace('(',' ').str.replace(')',' ').str.replace('/',' ')

/var/folders/3m/68621g4n7pd2p55cvf4c9tmh0000gn/T/ipykernel_17476/274413722.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  notes_csv['pn_history'] = notes_csv['pn_history'].str.replace(':','').str.replace('-',' ').str.replace(',',' ').str.replace('(',' ').str.replace(')',' ').str.replace('/',' ')
/var/folders/3m/68621g4n7pd2p55cvf4c9tmh0000gn/T/ipykernel_17476/274413722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_csv['pn_history'] = notes_csv['pn_history'].str.replace(':','').str.replace('-',' ').str.replace(',',' ').str.replace('(',' ').str.replace(')',' ').str.replace('/'

In [252]:
notes_csv.to_csv('/Users/suhancho/data/kaggle/nbme/notes.tsv',sep='\t',index=None)